# Method

### Import support packages

In [1]:
import os
import kagglehub
from DataGeneration import (
    get_season_mvp,
    collect_player_stats,
    normalize_player_stats,
    load_combined_mvp_2001_2023,
    fetch_mvp_stats_by_year
)


### make directories
player_data_path = 'Raw_player_data_sets'
os.makedirs(player_data_path, exist_ok=True)

### Fetch and save dataset

Yearly MVP results

In [ ]:


fetch_mvp_stats_by_year

Fetch dataset for each season